In [1]:

!pip install --upgrade transformers sentencepiece datasets sacrebleu rouge-score accelerate --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.5 MB/s eta 0:00:00


In [2]:
import sys
import pandas as pd
import torch
from datasets import Dataset, DatasetDict

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

print("Python:", sys.version.splitlines()[0])
print("Transformers version:", transformers.__version__)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Transformers version: 4.57.1


In [3]:
train_df = pd.read_csv("/content/train_te.csv")
val_df   = pd.read_csv("/content/val_te.csv")
test_df  = pd.read_csv("/content/test_te.csv")

print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)

# Autodetect columns (first two columns expected: input, response)
input_col  = train_df.columns[0]
target_col = train_df.columns[1]
print("Input column:", input_col)
print("Target column:", target_col)


Train: (1140, 8)
Val: (229, 8)
Test: (153, 8)
Input column: id
Target column: source


In [4]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

# Quick peek
print(dataset["train"][0])


{'id': 'dlg_cbb0155f815386275f372989ec609d9f', 'source': 'train.csv', 'type': 'dialogue', 'prompt': 'ఇది నేను ఆందోళన చెందాల్సిన విషయమా? నేను దాని గురించి ఏదైనా చేయాలా?', 'response': 'మీ కొడుకు జీవితంలోని ఇతర రంగాలు ఎలా పని చేస్తున్నాయి అనే దానిపై సమాధానం ఆధారపడి ఉంటుంది. అతను సంతోషంగా ఉన్నాడా లేదా సంతోషంగా ఉన్నాడా, అతను విశ్రాంతి సమయంలో ఒంటరిగా ఆడుతున్నాడా? విరామ సమయంలో అతను ఉన్న విద్యార్థులతో పాటు ఇతర సామాజిక సర్కిల్\u200cలలో అతనికి స్నేహితులు ఉన్నారా? అతని విద్యా పురోగతి ఎలా ఉంది? అతని క్లాస్\u200cమేట్స్\u200cలో అతని సామాజిక ఏకీకరణ ఎలా ఉంది? సమయం మరియు విరామ సమయంలో ఈ ఎంపికను తీసుకుంటుంది, లేదా పైన పేర్కొన్న ప్రాంతాలు అతనికి ఒత్తిడి లేదా ఇబ్బందిని చూపిస్తే మరియు వాటిని మరింత అర్థం చేసుకోవాలి మరియు నిర్వహించాలి.', 'lang': 'te', 'orig_id': None, 'fp': '88af44ce88510748816a9e7312ca4c8c'}


In [5]:
model_name = "google/mt5-small"   # good balance of size & multilingual ability

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Loaded model & tokenizer:", model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded model & tokenizer: google/mt5-small


In [6]:
MAX_LEN = 128

def preprocess(batch):
    # batch is a dict of lists
    inputs = tokenizer(batch[input_col], truncation=True, padding="max_length", max_length=MAX_LEN)
    labels = tokenizer(batch[target_col], truncation=True, padding="max_length", max_length=MAX_LEN)

    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_ds = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
print("Tokenized datasets:", {k: len(v) for k, v in tokenized_ds.items()})


Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Tokenized datasets: {'train': 1140, 'validation': 229, 'test': 153}


In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training size info (used to compute fallback eval/save steps)
train_size = len(tokenized_ds["train"])
batch_size = 4   # you can change this according to your GPU
num_epochs = 6   # reduce/increase based on your compute budget
print("train_size:", train_size, "batch_size:", batch_size, "num_epochs:", num_epochs)


train_size: 1140 batch_size: 4 num_epochs: 6


In [8]:
# Define batch_size and num_epochs
batch_size = 4    # You can increase it based on your GPU's memory capacity
num_epochs = 6    # You can adjust the number of epochs for training

# Define the output directory where model checkpoints will be saved
output_dir = "./te_chatbot_model"  # You can modify this path to your preference

# Simplified TrainingArguments for transformers version 4.57.1
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,          # Save model every 500 steps
    eval_steps=500,          # Evaluate every 500 steps
    save_total_limit=2       # Keep 2 checkpoints max
)

print("TrainingArguments set successfully for transformers v4.57.1")


TrainingArguments set successfully for transformers v4.57.1


In [9]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained model (mT5-small or any other suitable model)
model_name = "google/mt5-small"  # Replace with your model name if different
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Model loaded successfully.")


Model loaded successfully.


In [10]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your CSV files into pandas DataFrames
train_df = pd.read_csv("/content/train_te.csv")
val_df = pd.read_csv("/content/val_te.csv")
test_df = pd.read_csv("/content/test_te.csv")

# Check the shape of the data
print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

# Convert pandas DataFrames to HuggingFace datasets
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

# Check if dataset is loaded correctly
print(dataset)


Train shape: (1140, 8)
Validation shape: (229, 8)
Test shape: (153, 8)
DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 1140
    })
    validation: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 229
    })
    test: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 153
    })
})


In [11]:
from transformers import AutoTokenizer

# Load the tokenizer for mT5 (or your model)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
def preprocess(example):
    # Tokenize 'prompt' for the input text and 'response' for the target text
    inputs = tokenizer(example["prompt"], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(example["response"], truncation=True, padding='max_length', max_length=128)

    # Store the tokenized labels in 'labels'
    inputs["labels"] = labels["input_ids"]
    return inputs


In [13]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")  # Or your model

# Tokenize the dataset using the new column names ('prompt' and 'response')
def preprocess(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(example["response"], truncation=True, padding='max_length', max_length=128)

    inputs["labels"] = labels["input_ids"]
    return inputs

# Tokenize the dataset
tokenized_ds = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# Verify tokenized dataset (checking a sample)
print(tokenized_ds["train"][0])  # Verify tokenization


Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

{'input_ids': [3490, 6224, 15842, 4664, 2742, 100434, 18088, 3462, 59714, 139760, 34579, 19602, 291, 15842, 4664, 259, 69283, 43374, 20842, 8452, 18425, 39970, 119840, 14640, 291, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [13096, 15330, 175819, 259, 80660, 100915, 3490, 48361, 84694, 8248, 5336, 5862, 66657, 66899, 1

In [14]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
print("Model loaded successfully!")


Model loaded successfully!


In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [16]:
# Define batch_size and num_epochs
batch_size = 4
num_epochs = 6

output_dir = "./te_chatbot_model"

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2
)

print("TrainingArguments ready!")


TrainingArguments ready!


In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

print("Trainer initialized successfully!")


/tmp/ipython-input-2274696399.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer initialized successfully!


In [18]:

import os, sys, types

# Ensure Trainer won't try to use WANDB
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"

# Create a minimal fake wandb module that has the functions Trainer expects
fake_wandb = types.SimpleNamespace(
    init=lambda *a, **k: None,
    log=lambda *a, **k: None,
    finish=lambda *a, **k: None,
    run=types.SimpleNamespace(name="disabled"),
    config={},
)

# Inject it into sys.modules so imports will find it
sys.modules["wandb"] = fake_wandb

# Now remove the WandbCallback from the trainer (if it was added)
try:
    from transformers.integrations import WandbCallback
    trainer.remove_callback(WandbCallback)
except Exception:
    # ignore if not present or removal fails
    pass

print("WandB monkey-patched and WandbCallback removed (if present). Now run trainer.train() again.")


WandB monkey-patched and WandbCallback removed (if present). Now run trainer.train() again.


In [19]:
trainer.train()


Step,Training Loss
50,11.620700
100,5.419700
150,4.146000
200,3.693200
250,3.384100
300,3.315600
350,3.123500
400,3.172000
450,3.098900
500,3.055900


TrainOutput(global_step=1710, training_loss=3.235467113249483, metrics={'train_runtime': 739.3993, 'train_samples_per_second': 9.251, 'train_steps_per_second': 2.313, 'total_flos': 904161971404800.0, 'train_loss': 3.235467113249483, 'epoch': 6.0})

In [20]:
metrics = trainer.evaluate(tokenized_ds["test"])
metrics


{'eval_loss': 2.4858226776123047,
 'eval_runtime': 2.4969,
 'eval_samples_per_second': 61.276,
 'eval_steps_per_second': 15.619,
 'epoch': 6.0}

In [21]:
trainer.save_model("./final_telugu_chatbot")
tokenizer.save_pretrained("./final_telugu_chatbot")

print("Model saved successfully!")


Model saved successfully!


In [22]:
def te_chatbot(text):
    # Prepare inputs with max_length (REMOVE WARNINGS)
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128  # safe for mT5
    ).to(model.device)

    # Generate with repetition control
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        min_length=5,
        num_beams=4,
        no_repeat_ngram_size=3,        # IMPORTANT: reduces repetition
        repetition_penalty=2.0,        # stronger penalty
        length_penalty=1.0,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [23]:
print("Telugu Chatbot Ready! Type 'exit' to stop.\n")

while True:
    user_msg = input("మీరు: ")

    if user_msg.lower() in ["exit", "quit", "stop"]:
        print("Chatbot session ended.")
        break

    bot_reply = te_chatbot(user_msg)
    print("బాట్:", bot_reply)


Telugu Chatbot Ready! Type 'exit' to stop.

మీరు: నాతో ఉండాలనుకునే వారితో నేను సురక్షితమైన సంబంధాన్ని కోరుకుంటున్నాను మరియు దాని కోసం ఎవరు నిజంగా కృషి చేస్తారో. నేను అందుబాటులో లేని పురుషులు మరియు సాన్నిహిత్యం మరియు సంబంధం లేని వారి పట్ల ఆకర్షితుడయ్యాను. పురుషులు నన్ను శాసించటానికి మరియు నియంత్రించటానికి నేను అనుమతిస్తాను ఎందుకంటే వారు నన్ను నియంత్రిస్తున్నారని ఆరోపిస్తున్నారు. నేను పురుషులు మానసికంగా నన్ను దుర్వినియోగం చేయడానికి అనుమతిస్తాను మరియు నేను వారి బెక్ మరియు కాల్ వద్ద ఉన్నాను. నేను ఒంటరిగా ఉండటం లేదా ఒంటరిగా ఏదైనా చేయడం సౌకర్యంగా లేదు. నేను జీవించడానికి చుట్టూ ఉన్న వ్యక్తి యొక్క భద్రత అవసరమని నేను భావిస్తున్నాను. నేను ఏమి తప్పు చేస్తున్నానో నాకు తెలుసు మరియు పరిస్థితులు మారతాయని ఆశిస్తూ ఎలాగైనా చేస్తాను. ఈ ప్రవర్తన మరియు ఆలోచన ప్రక్రియను నేను ఎలా ఆపాలి?,"మీరు నా అత్యంత ఇటీవలి పోస్ట్ చదవడానికి ఆసక్తి కలిగి ఉండవచ్చు, సాన్నిహిత్యం మీతో ప్రారంభమవుతుంది. దానిలో నేను స్వీయ కనెక్షన్ మరియు ఆవిష్కరణ మార్గంలో ప్రారంభించడానికి మీకు సహాయం చేయడానికి 7 నిపుణుల చిట్కాలను అందిస్తున్నాను. మీ